In [1]:
#
# This notebook is written in a way intended to be view
# with cell inputs hidden. If this comment is visible,
# the data is at the end of the report, you may just ignore
# all the code.
#
# intended usage: convert to html with cell execution enabled
# > jupyter nbconvert --to=html --no-input --ExecutePreprocessor.enabled=True TM-status-report.ipynb
#
# if working in the gis repository, run:
# > make
# to automatically run, convert, and publish of github pages


# Tasking Manager COVID-19 projects summary


This is a Jupyter notebook for quickly summarizing the status of a HOTOSM Tasking Manager campaing using the TM API.


In [2]:
from datetime import datetime

# set the text search keyword
textSearch = 'covid'

instance = 'tasks.hotosm.org'

# for notebooks used for reporting, it's nice to
# timestamp of the data, since it can get old quickly
now = datetime.utcnow()
print(f'data last updated: {now} UTC')
print(f'tasking manager:   {instance}')
print(f'search keyword:    {textSearch}')

data last updated: 2020-04-19 08:34:28.682446 UTC
tasking manager:   tasks.hotosm.org
search keyword:    covid


In [3]:

import requests
import json
import pandas as pd

# make sure pandas doesn't truncate the dataframe when rendered
pd.set_option('display.max_rows', 500)

def getAllPages(textSearch):
    page = 1
    while True:
        url = f'https://{instance}/api/v1/project/search?textSearch={textSearch}&page={page}'
        r = requests.get(url, 
            headers = {
                'Accept-Language': '*',
                'Content-Type': 'application/json'
            }
        )
        j = json.loads(r.content)
        df = pd.DataFrame(json.loads(r.content)['results'])
        yield df
        if j['pagination']['hasNext'] is True:
            page = j['pagination']['nextNum']
        else:
            break

            
# concatenate paginated API results into one dataframe
result = pd.concat(getAllPages(textSearch), ignore_index=True);
# search API pagination sometimes returns the same project more than once
result = result.drop_duplicates(subset='projectId');



# Currently published projects

The table below shows the current status of all published projects that include the `covid` keyword.

Projects are sorted by project id, meaning they are in somewhat choronological order.

In [27]:
# this cells applyies some sematic style to make the
# search summary nicer to read
from matplotlib.colors import LinearSegmentedColormap

# colors from the HOTOSM media kit :)
hotosm_colors = {
  "red": "#D73F3F",
  "red-dark": "#6C2020",
  "red-light": "#FFEDED",
  "orange": "#FAA71E",
  "tan": "#F0EFEF",
  "blue-dark": "#2C3038",
  "blue-grey": "#68707F",
  "blue-light": "#929DB3",
  "grey-light": "#E1E0E0",
  "green": "#53AE62"
  }

# create a colormap for colorign mappedness and validatedness percentages
colors = [hotosm_colors['red'],hotosm_colors['orange'],hotosm_colors['green']]
colormap = LinearSegmentedColormap.from_list('hotosm-RdOrGr', colors, N=20)

# Pandas dataframe render styler functions
def highlight_priority(val):
    """set cell color based on proejct priority in TM"""
    lookup = {
        'LOW': hotosm_colors['grey-light'],
        'MEDIUM': hotosm_colors['green'],
        'HIGH': hotosm_colors['orange'],
        'URGENT': hotosm_colors['red']
    }
    color = lookup[val]
    return f'background-color: {color}'


def format_project_link(id):
    return f'https://{instance}/project/{id}'

def format_clickable_project_id(id):
    url = format_project_link(id)
    return f'<a href="{url}">{id}</a>'

# Total count of projects
print(f'total number of published projects: {len(result)}')

# We select only the most interesting fields from the dataframe
relevant = result[['projectId','name','percentMapped', 'percentValidated','priority','status', 'campaignTag']]

# Sorting by project id is somewhat cronological
# and also keeps the table order intact as new project are created
sorted_vals = relevant.sort_values('projectId')

# add a Tasking manager link
sorted_vals['hyperlink'] = sorted_vals['projectId'].apply(format_project_link)


# write the table out as CSV, so it will be accessible e.g. gy google spreadsheets
# index=False removes the pandas internal row index column from the CSV
sorted_vals.to_csv('TM-status-report.csv', index=False)

# the hyperlink text is just for the CSV
# notebook projectId is clickable, so the hyperlink is redundant
del sorted_vals['hyperlink']

# apply custom styler functions to final table
styled = sorted_vals.style\
    .background_gradient(cmap=colormap,subset=['percentMapped', 'percentValidated'])\
    .applymap(highlight_priority, subset=['priority'])\
    .format({'projectId': format_clickable_project_id})


display(styled)

total number of published projects: 56


,projectId,name,percentMapped,percentValidated,priority,status,campaignTag
55,5909,"COVID-19, Quehue, Anna, Cusco Peru",98,26,LOW,PUBLISHED,COVID-19
54,7372,"COVID-19 - Paucartambo 1, Cusco, Peru",99,7,LOW,PUBLISHED,COVID-19
53,7721,"COVID-19 - Coya - Calca, Peru",98,98,LOW,PUBLISHED,COVID-19
52,7768,"COVID-19 - Chamaca - Chumbivilcas, Peru",96,99,LOW,PUBLISHED,COVID-19
51,7770,"COVID-19 San Jeronimo - Cusco , Peru",97,59,LOW,PUBLISHED,COVID-19
5,7968,"COVID-19 - Andahuaylillas - Quispicanchi, Peru",90,79,HIGH,PUBLISHED,COVID-19
50,7970,"COVID-19 Lucre - Quispicanchi, Peru",100,100,LOW,PUBLISHED,COVID-19
49,8049,"COVID-19 Omacha - Paruro, Peru",98,59,LOW,PUBLISHED,COVID-19
48,8159,"Impendle, South Africa COVID-19",83,21,LOW,PUBLISHED,None
0,8188,Mapatón COVID-19 Medellín - Comuna 1,91,8,URGENT,PUBLISHED,None


Notebook maintained by @jarmokivekas (hotosm slack), feedback and development ideas welcome.